<a href="https://colab.research.google.com/github/randywreed/sav2csv/blob/master/Clean_sav_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install voila
!jupyter serverextension enable voila —sys-prefix


     |████████████████████████████████| 6.9MB 2.4MB/s 
     |████████████████████████████████| 215kB 28.3MB/s 
     |████████████████████████████████| 112kB 37.8MB/s 
     |████████████████████████████████| 552kB 43.2MB/s 
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
  Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1


Enabling: voila
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?
Enabling: —sys-prefix
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension —sys-prefix
      X is —sys-prefix importable?


In [1]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!pip install pyreadstat
import pandas as pd

     |████████████████████████████████| 2.0MB 2.8MB/s 


In [3]:
import ipywidgets as widgets

In [4]:
from IPython.display import display

In [5]:
upload=widgets.FileUpload(accept=".sav",multiple=False)
upload

FileUpload(value={}, accept='.sav', description='Upload')

In [6]:
with open (upload.metadata[0]['name'],"w+b") as i:
  i.write(upload.data[0])

In [7]:
import pyreadstat
df,meta=pyreadstat.read_sav(upload.metadata[0]['name'])

In [8]:
df.colmnns=meta.column_names

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [9]:
klab={}
klab=meta.column_names_to_labels
#type(klab)

dict

In [10]:
labs=[]
for k,v in klab.items():
  #print("{}:{}".format(k,v))
  labs.append([k,v])

In [11]:
df_enriched=pyreadstat.set_value_labels(df,meta,formats_as_category=True)

In [12]:
def strip_it(initialsent):
  import string, re
  en_stops=set(stopwords.words('english'))
  #update stopwords
  en_stops.remove('not')
  newstopwords=['what','why','who','when','where','how','many']
  en_stops.update(newstopwords)
  #remove puntuation, stopwords, and add spaces to runon words 
  translator = str.maketrans('', '', string.punctuation)
  newvar=initialsent.translate(translator)
  #print('after punctuation strip=',newvar)
  newvar=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', newvar)
  #print('add spaces if necessary=',newvar)
  c_split=newvar.split(" ")
  c_strip=""
  c_stripns=""
  for word in c_split:
    if str.lower(word) not in en_stops:
      c_strip=c_strip+word+" "
      c_stripns=c_stripns+word+"_"
  c_stripns=c_stripns.rstrip('_')
  return (c_stripns,c_strip)


In [13]:
def create_options(lab,val):
  out=[]
  ret=strip_it(val)
  out=[lab,ret[0],ret[1],"other"]
  return out

In [14]:
t=-1


In [112]:
t+=1
if t=len(labs):
  print('End')
else:
  lab=labs[t]
  if lab[1] is None:
    lab[1]=lab[0]
  out=create_options(lab[0],lab[1])
  print(lab[1])
  labchg=widgets.Select(options=out,value='other',description=lab[0]+" Label Options:",disabled=False)
  wother=widgets.Text(value="New label",placeholder="Type Something",description="Other:",disabled=False)
  button=widgets.Button(description="Submit")
  edit=widgets.Button(description="Edit")
  output=widgets.Output()
  display(labchg,button,edit,output)
  def on_button_clicked(b):
    if labchg.value=="other":
        display(wother)
    else:
      with output:
        labs[t][1]=labchg.value
        print('label l={}'.format(labs[t][1]))
      
  def on_edit_clicked(e):
    wother.value=labchg.value
    display(wother)

  edit.on_click(on_edit_clicked)
  button.on_click(on_button_clicked)
  def on_save_clicked(s):
    with output:
      labs[t][1]=wother.value
      print('label w={}'.format(labs[t][1]))
      print("new label saved")
  wother.on_submit(on_save_clicked)



IndexError: ignored

In [113]:
col=[]
for c in labs:
  col.append(c[1])
df_enriched.columns=col


In [120]:
import os
base=os.path.basename(upload.metadata[0]['name'])
newfile=os.path.splitext(base)
#print(newfile[0])
df_enriched.to_csv(newfile[0]+".csv")

In [121]:
from IPython.display import FileLink
# Import a module to delete the file
import os
# Create a download function
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv within a Jupyter notebook

    Parameters
    ----------
    df : pandas data frame
    csv_file_name : str
    delete_prompt : bool
    """
    df.to_csv(csv_file_name, index=False)
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        os.remove(csv_file_name)


In [122]:
if 'google.colab' in str(get_ipython()):
  from google.colab import files
  files.download(newfile[0]+'.csv')
else:
  csv_download_link(df,newfile[0]+'.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>